In [ ]:
import aurai.data_source as ds
# 根据dsId获取数据源信息
dsId="5cb92eda46e0fb000e6f539f"
dsCycle="yyyyMMdd"
ds.describeDS(dsId)

# 载入数据
data = ds.load(dsId)
#print(data)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn. import RandomForestClassifier
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.preprocessing import PolynomialFeatures

# input parameters:
# tbName: 一个dataframe
# catCols: 一个list，存储类别型变量
# colDrop：一个list，存储型变量
def dataTransform(tbName=None,catCols=None ,colDrop=None):
    if isinstance(tbName,pd.DataFrame):
        cols = tbName.columns
        if isinstance(colDrop,list):
            cols=list(set(cols).difference(set(colDrop)))
            if isinstance(catCols,list):
                catCols=list(set(catCols).difference(set(colDrop)))
                conCols=list(set(cols).difference(set(catCols)))
                
                 # 针对连续型变量，将所有值转换成float型
                for conCol in conCols:
                    tbName[conCol]=tbName[conCol].map(lambda i: np.nan if(i=='\\N' or i=='NaN' or i=='nan' ) else i).astype(np.dtype('float64'))
                data_con = tbName[conCols].fillna(0)
                  # 针对类别型变量，做one-hot编码处理
                for subCol in catCols:
                    subDummies = pd.get_dummies(tbName[subCol], prefix=subCol)
                    data_con = data_con.join(subDummies)
                    
                data_con_cat = data_con
                
                return data_con_cat
        else:
            raise ValueError('colDrop should be a list!')


#data_x : Training data
#degree : The degree of the polynomial features. integer
def data_transform_polynomial(data_x,degree):
    if degree==1:
        X_redegree = data_x
    else:
        poly = preprocessing.PolynomialFeatures(degree)
        X_redegree = poly.fit_transform(data_x)
    return {'x' : X_redegree}


#transform_type : datatransform such as 'Smote' 'Naive' 'Woe'
#smote_ratio :  the keys correspond to the targeted classes. The values correspond to the desired number of samples.
#data_x : Training data
#data_y : Target values
def data_transform_oversampling(transform_type,smote_ratio,data_x,data_y):
    if transform_type=='Smote':
        nv=sum(data_y==False)
        pv = round(nv * smote_ratio)
        ratio = {False : nv ,True : pv }
        X_resampled, y_resampled = SMOTE(ratio = ratio).fit_sample(data_x, data_y)
    elif transform_type=='Naive':
        nv=sum(data_y==False)
        pv = round(nv * smote_ratio)
        ratio = {False : nv ,True : pv }
        ros = RandomOverSampler(ratio = ratio,random_state=0)
        X_resampled, y_resampled = ros.fit_sample(data_x, data_y)
    else:
        X_resampled, y_resampled = data_x, data_y
    return {'x' : X_resampled, 'y' : y_resampled}


# 切分训练集与测试集
# input：
def getTrainTestSample(df, taregt, test_size=0.5):
    data_cols = list(df.columns)
    train_data, test_data = train_test_split(df, test_size=test_size)

    # 获取feature与target
    target = taregt.upper()
    data_cols.remove(target)
    feature = data_cols

    train_X = train_data[feature]
    train_y = train_data[target]
    test_X = test_data[feature]
    test_y = test_data[target]
    return train_X, train_y, test_X, test_y

# get import feature
# inputs:
#        model 【】
#                  model = RandomForestClassifier()
#        train_feature 【a dataframe,which contains the feature to be trained】
#        train_target 【a dataframe,which contains the target to be trained】
#        fi_threshold 【a value from 0-100,to control the threshold to be used for training】
# output: featureImporColumn 【importance column list according to the importance value】
#         featureImporValue   【importance feature value according to the value】
def getImpFeature(model, train_X, train_y, fi_threshold=10):
    model.fit(train_X, train_y)
    feature_importance = model.feature_importances_
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    important_idx = np.where(feature_importance > fi_threshold)[0]
    sorted_idx = np.argsort(feature_importance[important_idx])[::-1]
    impFeatureValues = feature_importance[important_idx[sorted_idx]]
    impFeatureCols = train_X.iloc[:,important_idx[sorted_idx]].columns
    return impFeatureCols,impFeatureValues

def visualImporFeature():
    pass

def modelTrain(model,parameters,train_X,train_y,cv=5,scoring='f1'):
    clf = GridSearchCV(estimator=model,param_grid=parameters,cv=cv,scoring=scoring)
    clf.fit(X=train_X,y=train_y)
    return clf


def plotROC(true_label,predict_prob,roc_auc):
    fpr, tpr, thresholds = metrics.roc_curve(true_label, predict_prob)
    plt.figure()
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label='RF(area = {0:0.2f})'
         ''.format(roc_auc))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()


# 对预测pro进行切断来看结果 
def calBinPredProb(true_label,predict_prob):
    temp=pd.DataFrame({'true_label':true_label,'predict_prob':predict_prob})
    bins=np.arange(0,1.1,0.1)
    temp['predict_prob_seg']=pd.cut(temp['predict_prob'],bins=bins)
    return temp[['true_label','predict_prob_seg']].groupby('predict_prob_seg').aggregate(['count','sum'])

    
#true_label : the true label 
#predict_prob : thre predicted prob
#thre: the threshold to seperate 0 and 1
def model_evaluate(true_label,predict_prob,thre):
    predict_label = np.where(predict_prob>thre,1,0)
    confusion_matrix = metrics.confusion_matrix(true_label, predict_label)
    
    accuracy =metrics.accuracy_score(true_label, predict_label)    
    precision = metrics.precision_score(true_label, predict_label)
    recall = metrics.recall_score(true_label, predict_label)
    f1 = metrics.f1_score(true_label, predict_label)
    fpr, tpr, thresholds = metrics.roc_curve(true_label, predict_prob)
    roc_auc = metrics.auc(fpr, tpr)
    prc = metrics.average_precision_score(true_label,predict_prob)
    
    
    print('ROC图如下：')
    plotROC(true_label,predict_prob,roc_auc)
    
    print("confusion matrix:")
    print("------------------------- ")
    print("| TP: %5d | FP: %5d |" % (confusion_matrix[1, 1], confusion_matrix[0, 1]))
    print("----------------------- ")
    print("| FN: %5d | TN: %5d |" % (confusion_matrix[1, 0], confusion_matrix[0, 0]))
    print(" ------------------------- ")
    
    #print('ROC图如下：')
    #true_label_len= len(true_label)
    #for i in range(0,true_label_len):
    #    plot.plot(fpr, tpr, lw=1, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))
    
    print('预测结果分段统计，预测值与实际命中情况统计')
    print(calBinPredProb(true_label,predict_prob))
    
    return {'accuracy':round(accuracy,4),'precision':round(precision,4),'recall':round(recall,4),'f1':round(f1,4),'roc_auc':roc_auc,'prc':prc}

In [ ]:
###########################################################################
# 数据预处理
########################################################################### 
tbName=data
catCols =[
'user_id' # 用户编号
,'bill_no' # 手机号码
,'city_id' # 城市
,'sex_new' # 性别
,'origin' # 籍贯
,'real_name_flag_new' # 实名认证标识
,'vpmn_flag' # 是否虚拟网成员
,'gpr_memb_flag' # 是否集团网成员
,'vip_flag' # 是否大客户
,'upay_flag' # 统一支付用户标识
,'famstru' # 家庭结构
,'occu' # 职业
,'sub_occu_name' # 细分的职业
,'life_stage' # 人生阶段
,'if_salariat' # 是否工薪阶层
,'edu_level' # 教育水平
,'stay_city_name_new' # 居住城市
,'work_city_name_new' # 工作城市
,'stay_town_flag' # 是否城镇居民
,'fir_imei_brand' # 排名第一终端品牌
,'new_imei_flag' # 新手机标识
,'drv_flag' # 是否司机
]
colDrop =  [
'user_id'
,'bill_no'
,'city_id'
,'car_flag'
,'train_test_flag'
,'wz_sms_cnt' # 近一年违章短信接收次数
]

# 对数据进行处理，连续值中Nan转换为0，离散值转换为独热编码
data_X = dataTransform(tbName=tbName,catCols=catCols ,colDrop=colDrop)

In [ ]:
data_X_y=pd.concat([data_X,data[['car_flag','train_test_flag']]],axis=1)
data_X_y.shape

In [ ]:
###########################################################################
#切分训练集与测试集  
###########################################################################
cols = list(data_X_y.columns)
cols.remove('car_flag')
cols.remove('train_test_flag')
data_train = data_X_y[data_X_y['train_test_flag']==1]
data_test = data_X_y[data_X_y['train_test_flag']==0]
#data_train_X, data_test_X, data_train_y, data_test_y = train_test_split(data_X,data[['car_flag']], test_size=0.5, random_state=0)

data_train_X = data_train[cols]
data_train_y = data_train['car_flag']
data_test_X = data_test[cols]
data_test_y = data_test['car_flag']

In [ ]:
###########################################################################
# 获取显著性特征  
########################################################################### 
# n_estimators 基评估器的数量
model = RandomForestClassifier(n_estimators=300)
fi_threshold = 5
impFeatureCols,impFeatureValues = getImpFeature(model=model, train_X=data_train_X, train_y=np.ravel(data_train_y.values), fi_threshold=fi_threshold)

pd.DataFrame({'impFeatureCols':impFeatureCols,'impFeatureValues':impFeatureValues})
pd.DataFrame({'impFeatureCols':impFeatureCols,'impFeatureValues':impFeatureValues}).to_csv(r'importFeatures.csv',index=False,encoding='utf-8')

In [ ]:
impFeatureCols=pd.read_csv(r'importFeatures.csv')['impFeatureCols']
impFeatureCols
print()

In [ ]:
###########################################################################
# 模型训练, 调参、评估 
########################################################################### 
model = RandomForestClassifier(min_samples_leaf=50)
parameters = {
			  'n_estimators':[100,200,300,400,500]
			 }
train_X = data_train[impFeatureCols]
train_y = data_train['car_flag']
cv = 5
scoring='f1'
clf = modelTrain(model,parameters,train_X,train_y,cv=5,scoring='f1')

In [ ]:
train_X=data_train_X[impFeatureCols]
train_y=data_train['car_flag']

#model = RandomForestClassifier(min_samples_leaf=50,n_estimators=200)
#clf = model.fit(train_X,train_y)
joblib.dump(clf,'rf.pkl')
clf=joblib.load('rf.pkl')

In [ ]:
data_train_predict_y = clf.predict_proba(X=train_X)[:,1]
evalTrain = model_evaluate(true_label=train_y,predict_prob=data_train_predict_y,thre=0.5)
evalTrain

In [ ]:
test_X = data_test_X[impFeatureCols]
test_y = data_test['car_flag']

In [ ]:
clf=joblib.load('rf.pkl')
test_X = data_test_X[impFeatureCols]
test_y = data_test['car_flag']
data_test_predict_y = clf.predict_proba(X=test_X)[:,1]

#evalTest = model_evaluate(true_label=test_y,predict_prob=data_test_predict_y,thre=0.5)
#evalTest

In [ ]:
evalTest = model_evaluate(true_label=test_y,predict_prob=data_test_predict_y,thre=0.5)
evalTest